In [1]:
import pandas as pd
import numpy as np

### Step 2.1: Parse and clean

In [2]:
df1 = pd.read_csv('client_data (1).csv', parse_dates=["date_activ","date_end","date_modif_prod","date_renewal"])
df2 = pd.read_csv('price_data (1).csv', parse_dates=["price_date"])

In [3]:
df1.head()

,id,channel_sales,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,...,has_gas,imp_cons,margin_gross_pow_ele,margin_net_pow_ele,nb_prod_act,net_margin,num_years_antig,origin_up,pow_max,churn
0,24011ae4ebbe3035111d65fa7c15bc57,foosdfpfkusacimwkcsosbicdxkicaua,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,...,t,0.00,25.44,25.44,2,678.99,3,lxidpiddsbxsbosboudacockeimpuepw,43.648,1
1,d29c2c54acc38ff3c0614d0a653813dd,MISSING,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,...,f,0.00,16.38,16.38,1,18.89,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.800,0
2,764c75f661154dac3a6c254cd082ea7d,foosdfpfkusacimwkcsosbicdxkicaua,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,...,f,0.00,28.60,28.60,1,6.60,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.856,0
3,bba03439a292a1e166f80264c16191cb,lmkebamcaaclubfxadlmueccxoimlema,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,...,f,0.00,30.22,30.22,1,25.46,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,13.200,0
4,149d57cf92fc41cf94415803a877cb4b,MISSING,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,...,f,52.32,44.91,44.91,1,47.98,6,kamkkxfxxuwbdslkwifmmcsiusiuosws,19.800,0


In [4]:
df1.shape

(14606, 26)

In [5]:
df2.head()

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0
3,038af19179925da21a25619c5a24b745,2015-04-01,0.149626,0.0,0.0,44.266931,0.0,0.0
4,038af19179925da21a25619c5a24b745,2015-05-01,0.149626,0.0,0.0,44.266931,0.0,0.0


In [6]:
df2.shape

(193002, 8)

In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              14606 non-null  object        
 1   channel_sales                   14606 non-null  object        
 2   cons_12m                        14606 non-null  int64         
 3   cons_gas_12m                    14606 non-null  int64         
 4   cons_last_month                 14606 non-null  int64         
 5   date_activ                      14606 non-null  datetime64[ns]
 6   date_end                        14606 non-null  datetime64[ns]
 7   date_modif_prod                 14606 non-null  datetime64[ns]
 8   date_renewal                    14606 non-null  datetime64[ns]
 9   forecast_cons_12m               14606 non-null  float64       
 10  forecast_cons_year              14606 non-null  int64         
 11  fo

In [8]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193002 entries, 0 to 193001
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   id                  193002 non-null  object        
 1   price_date          193002 non-null  datetime64[ns]
 2   price_off_peak_var  193002 non-null  float64       
 3   price_peak_var      193002 non-null  float64       
 4   price_mid_peak_var  193002 non-null  float64       
 5   price_off_peak_fix  193002 non-null  float64       
 6   price_peak_fix      193002 non-null  float64       
 7   price_mid_peak_fix  193002 non-null  float64       
dtypes: datetime64[ns](1), float64(6), object(1)
memory usage: 11.8+ MB


### Step 2.2: Time-based features

In [9]:
ref_date = pd.to_datetime("2016-12-31")  

df1["contract_age_days"] = (ref_date - df1["date_activ"]).dt.days
df1["days_to_end"] = (df1["date_end"] - ref_date).dt.days
df1["days_since_modif"] = (ref_date - df1["date_modif_prod"]).dt.days
df1["days_to_renewal"] = (df1["date_renewal"] - ref_date).dt.days
df1["contract_ends_soon"] = (df1["days_to_end"] < 90).astype(int)

### Step 2.3: Consumption patterns 

In [10]:
df1["avg_monthly_cons"] = df1["cons_12m"] / 12
df1["last_month_vs_avg"] = df1["cons_last_month"] / (df1["avg_monthly_cons"] + 1e-6)
df1["cons_ratio_vs_forecast"] = df1["cons_12m"] / (df1["forecast_cons_12m"] + 1e-6)
df1["gas_to_elec_ratio"] = df1["cons_gas_12m"] / (df1["cons_12m"] + 1)
df1["has_gas_flag"] = (df1["has_gas"] == "t").astype(int)

### Step 2.4: Margin & profitability features

In [11]:
df1["margin_ratio"] = df1["margin_net_pow_ele"] / (df1["margin_gross_pow_ele"] + 1e-6)
df1["profit_per_product"] = df1["net_margin"] / (df1["nb_prod_act"] + 1)
df1["profitability_index"] = df1["net_margin"] / (df1["cons_12m"] + 1)

### Step 2.5: Power & usage flags

In [12]:
df1["is_high_power"] = (df1["pow_max"] > df1["pow_max"].median()).astype(int)
df1["is_low_usage"] = (df1["cons_12m"] < df1["cons_12m"].median()).astype(int)

### Step 2.6: Merge with price data

In [13]:
latest_price = df2.sort_values(["id","price_date"]).groupby("id").tail(1)
latest_price = latest_price.set_index("id").add_prefix("latest_")
df1 = df1.set_index("id").join(latest_price, how="left").reset_index()

In [14]:
df1= df1.drop(columns=['id'])

### Step 2.7: Price-related features

In [15]:
df1["price_peak_diff"] = df1["forecast_price_energy_peak"] - df1["latest_price_peak_var"]
df1["price_offpeak_diff"] = df1["forecast_price_energy_off_peak"] - df1["latest_price_off_peak_var"]

df1["avg_price_diff"] = (df1["price_peak_diff"] + df1["price_offpeak_diff"]) / 2

### Step 2.8: Encoded categorical variables 

In [16]:
print(df1.columns.tolist())

['channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'date_activ', 'date_end', 'date_modif_prod', 'date_renewal', 'forecast_cons_12m', 'forecast_cons_year', 'forecast_discount_energy', 'forecast_meter_rent_12m', 'forecast_price_energy_off_peak', 'forecast_price_energy_peak', 'forecast_price_pow_off_peak', 'has_gas', 'imp_cons', 'margin_gross_pow_ele', 'margin_net_pow_ele', 'nb_prod_act', 'net_margin', 'num_years_antig', 'origin_up', 'pow_max', 'churn', 'contract_age_days', 'days_to_end', 'days_since_modif', 'days_to_renewal', 'contract_ends_soon', 'avg_monthly_cons', 'last_month_vs_avg', 'cons_ratio_vs_forecast', 'gas_to_elec_ratio', 'has_gas_flag', 'margin_ratio', 'profit_per_product', 'profitability_index', 'is_high_power', 'is_low_usage', 'latest_price_date', 'latest_price_off_peak_var', 'latest_price_peak_var', 'latest_price_mid_peak_var', 'latest_price_off_peak_fix', 'latest_price_peak_fix', 'latest_price_mid_peak_fix', 'price_peak_diff', 'price_offpeak_diff', 'avg_pri

In [17]:
df1.shape

(14606, 50)

In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 50 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   channel_sales                   14606 non-null  object        
 1   cons_12m                        14606 non-null  int64         
 2   cons_gas_12m                    14606 non-null  int64         
 3   cons_last_month                 14606 non-null  int64         
 4   date_activ                      14606 non-null  datetime64[ns]
 5   date_end                        14606 non-null  datetime64[ns]
 6   date_modif_prod                 14606 non-null  datetime64[ns]
 7   date_renewal                    14606 non-null  datetime64[ns]
 8   forecast_cons_12m               14606 non-null  float64       
 9   forecast_cons_year              14606 non-null  int64         
 10  forecast_discount_energy        14606 non-null  float64       
 11  fo

In [19]:
df1.isnull().sum()

channel_sales                     0
cons_12m                          0
cons_gas_12m                      0
cons_last_month                   0
date_activ                        0
date_end                          0
date_modif_prod                   0
date_renewal                      0
forecast_cons_12m                 0
forecast_cons_year                0
forecast_discount_energy          0
forecast_meter_rent_12m           0
forecast_price_energy_off_peak    0
forecast_price_energy_peak        0
forecast_price_pow_off_peak       0
has_gas                           0
imp_cons                          0
margin_gross_pow_ele              0
margin_net_pow_ele                0
nb_prod_act                       0
net_margin                        0
num_years_antig                   0
origin_up                         0
pow_max                           0
churn                             0
contract_age_days                 0
days_to_end                       0
days_since_modif            

In [20]:
df1.select_dtypes(include='object').columns.tolist()

['channel_sales', 'has_gas', 'origin_up']

### Below need to be checked

In [21]:
for col in ['channel_sales', 'has_gas', 'origin_up']:
    print(f"\n--- {col} ---")
    print(df1[col].unique())


--- channel_sales ---
['foosdfpfkusacimwkcsosbicdxkicaua' 'MISSING'
 'lmkebamcaaclubfxadlmueccxoimlema' 'usilxuppasemubllopkaafesmlibmsdf'
 'ewpakwlliwisiwduibdlfmalxowmwpci' 'epumfxlbckeskwekxbiuasklxalciiuu'
 'sddiedcslfslkckwlfkdpoeeailfpeds' 'fixdbufsefwooaasfcxdxadsiekoceaa']

--- has_gas ---
['t' 'f']

--- origin_up ---
['lxidpiddsbxsbosboudacockeimpuepw' 'kamkkxfxxuwbdslkwifmmcsiusiuosws'
 'ldkssxwpmemidmecebumciepifcamkci' 'MISSING'
 'usapbepcfoloekilkwsdiboslwaxobdp' 'ewxeelcelemmiwuafmddpobolfuxioce']


### In channel_sales column and original_up, replace MISSING category by mode 

In [ ]:
# df1['channel_sales'] = df1['channel_sales'].replace('MISSING', df1['channel_sales'].mode()[0])

In [ ]:
# df1['origin_up'] = df1['origin_up'].replace('MISSING', df1['origin_up'].mode()[0])

In [22]:
df1= df1.drop(columns=['channel_sales'])

In [23]:
df1= df1.drop(columns=['origin_up'])

In [ ]:
# from sklearn.preprocessing import LabelEncoder

In [ ]:
# label_enc = LabelEncoder()

# df1['channel_sales'] = label_enc.fit_transform(df1['channel_sales'])
# df1['has_gas'] = label_enc.fit_transform(df1['has_gas'])
# df1['origin_up'] = label_enc.fit_transform(df1['origin_up'])


In [ ]:
# df1_encoded = pd.get_dummies(df1, columns=['channel_sales', 'has_gas', 'origin_up'], drop_first=True)

In [25]:
df1.select_dtypes(include='object').columns.tolist()

['has_gas']

In [26]:
df1.shape

(14606, 48)

In [27]:
df1.head(10)

,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,...,latest_price_date,latest_price_off_peak_var,latest_price_peak_var,latest_price_mid_peak_var,latest_price_off_peak_fix,latest_price_peak_fix,latest_price_mid_peak_fix,price_peak_diff,price_offpeak_diff,avg_price_diff
0,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,...,2015-12-01,0.146033,0.085483,0.000000,44.266930,0.00000,0.000000,0.012659,-0.031552,-0.009447
1,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,...,2015-12-01,0.147600,0.000000,0.000000,44.444710,0.00000,0.000000,0.000000,-0.001889,-0.000945
2,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,...,2015-12-01,0.167798,0.088409,0.000000,44.444710,0.00000,0.000000,-0.000510,-0.002004,-0.001257
3,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,...,2015-12-01,0.148586,0.000000,0.000000,44.444710,0.00000,0.000000,0.000000,-0.001892,-0.000946
4,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,...,2015-12-01,0.119906,0.101673,0.073719,40.728885,24.43733,16.291555,-0.001658,-0.003006,-0.002332
5,8302,0,1998,2011-12-09,2016-12-09,2015-11-01,2015-12-10,796.94,1998,0.0,...,2015-12-01,0.163659,0.084587,0.000000,44.266930,0.00000,0.000000,0.001544,0.001116,0.001330
6,45097,0,0,2011-12-02,2016-12-02,2011-12-02,2015-12-03,8069.28,0,0.0,...,2015-12-01,0.163361,0.084305,0.000000,44.266930,0.00000,0.000000,0.003233,0.002817,0.003025
7,29552,0,1260,2010-04-21,2016-04-21,2010-04-21,2015-04-22,864.73,751,0.0,...,2015-12-01,0.118175,0.100491,0.074516,40.728885,24.43733,16.291555,-0.001654,-0.003001,-0.002328
8,2962,0,0,2011-09-23,2016-09-23,2011-09-23,2015-09-25,444.38,0,0.0,...,2015-12-01,0.147600,0.000000,0.000000,44.444710,0.00000,0.000000,0.000000,-0.001889,-0.000945
9,26064,0,2188,2010-05-04,2016-05-04,2015-04-29,2015-05-05,2738.10,2188,0.0,...,2015-12-01,0.118755,0.101071,0.075096,40.728885,24.43733,16.291555,-0.001652,-0.002994,-0.002323


In [28]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 48 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   cons_12m                        14606 non-null  int64         
 1   cons_gas_12m                    14606 non-null  int64         
 2   cons_last_month                 14606 non-null  int64         
 3   date_activ                      14606 non-null  datetime64[ns]
 4   date_end                        14606 non-null  datetime64[ns]
 5   date_modif_prod                 14606 non-null  datetime64[ns]
 6   date_renewal                    14606 non-null  datetime64[ns]
 7   forecast_cons_12m               14606 non-null  float64       
 8   forecast_cons_year              14606 non-null  int64         
 9   forecast_discount_energy        14606 non-null  float64       
 10  forecast_meter_rent_12m         14606 non-null  float64       
 11  fo

In [30]:
df1.head()

,cons_12m,cons_gas_12m,cons_last_month,date_activ,date_end,date_modif_prod,date_renewal,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,...,latest_price_date,latest_price_off_peak_var,latest_price_peak_var,latest_price_mid_peak_var,latest_price_off_peak_fix,latest_price_peak_fix,latest_price_mid_peak_fix,price_peak_diff,price_offpeak_diff,avg_price_diff
0,0,54946,0,2013-06-15,2016-06-15,2015-11-01,2015-06-23,0.00,0,0.0,...,2015-12-01,0.146033,0.085483,0.000000,44.266930,0.00000,0.000000,0.012659,-0.031552,-0.009447
1,4660,0,0,2009-08-21,2016-08-30,2009-08-21,2015-08-31,189.95,0,0.0,...,2015-12-01,0.147600,0.000000,0.000000,44.444710,0.00000,0.000000,0.000000,-0.001889,-0.000945
2,544,0,0,2010-04-16,2016-04-16,2010-04-16,2015-04-17,47.96,0,0.0,...,2015-12-01,0.167798,0.088409,0.000000,44.444710,0.00000,0.000000,-0.000510,-0.002004,-0.001257
3,1584,0,0,2010-03-30,2016-03-30,2010-03-30,2015-03-31,240.04,0,0.0,...,2015-12-01,0.148586,0.000000,0.000000,44.444710,0.00000,0.000000,0.000000,-0.001892,-0.000946
4,4425,0,526,2010-01-13,2016-03-07,2010-01-13,2015-03-09,445.75,526,0.0,...,2015-12-01,0.119906,0.101673,0.073719,40.728885,24.43733,16.291555,-0.001658,-0.003006,-0.002332


### Tenure / Age of the customer

In [31]:
df1['tenure_days'] = (df1['date_end'] - df1['date_activ']).dt.days

### Time since last modification

In [32]:
df1['days_since_modif'] = (df1['date_end'] - df1['date_modif_prod']).dt.days

### Time until/after renewal

In [33]:
df1['days_to_renewal'] = (df1['date_renewal'] - df1['date_end']).dt.days

### Dropping original time stamp variables

In [34]:
df1 = df1.drop(columns=['date_activ', 'date_end', 'date_modif_prod', 'date_renewal'])

In [35]:
df1.head()

,cons_12m,cons_gas_12m,cons_last_month,forecast_cons_12m,forecast_cons_year,forecast_discount_energy,forecast_meter_rent_12m,forecast_price_energy_off_peak,forecast_price_energy_peak,forecast_price_pow_off_peak,...,latest_price_off_peak_var,latest_price_peak_var,latest_price_mid_peak_var,latest_price_off_peak_fix,latest_price_peak_fix,latest_price_mid_peak_fix,price_peak_diff,price_offpeak_diff,avg_price_diff,tenure_days
0,0,54946,0,0.00,0,0.0,1.78,0.114481,0.098142,40.606701,...,0.146033,0.085483,0.000000,44.266930,0.00000,0.000000,0.012659,-0.031552,-0.009447,1096
1,4660,0,0,189.95,0,0.0,16.27,0.145711,0.000000,44.311378,...,0.147600,0.000000,0.000000,44.444710,0.00000,0.000000,0.000000,-0.001889,-0.000945,2566
2,544,0,0,47.96,0,0.0,38.72,0.165794,0.087899,44.311378,...,0.167798,0.088409,0.000000,44.444710,0.00000,0.000000,-0.000510,-0.002004,-0.001257,2192
3,1584,0,0,240.04,0,0.0,19.83,0.146694,0.000000,44.311378,...,0.148586,0.000000,0.000000,44.444710,0.00000,0.000000,0.000000,-0.001892,-0.000946,2192
4,4425,0,526,445.75,526,0.0,131.73,0.116900,0.100015,40.606701,...,0.119906,0.101673,0.073719,40.728885,24.43733,16.291555,-0.001658,-0.003006,-0.002332,2245


In [36]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 45 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   cons_12m                        14606 non-null  int64         
 1   cons_gas_12m                    14606 non-null  int64         
 2   cons_last_month                 14606 non-null  int64         
 3   forecast_cons_12m               14606 non-null  float64       
 4   forecast_cons_year              14606 non-null  int64         
 5   forecast_discount_energy        14606 non-null  float64       
 6   forecast_meter_rent_12m         14606 non-null  float64       
 7   forecast_price_energy_off_peak  14606 non-null  float64       
 8   forecast_price_energy_peak      14606 non-null  float64       
 9   forecast_price_pow_off_peak     14606 non-null  float64       
 10  has_gas                         14606 non-null  object        
 11  im

In [37]:
df1.to_csv("final_processed_clients.csv", index=False)
print("✅ Saved processed_clients.csv")

✅ Saved processed_clients.csv
